In [1]:
import tensorflow as tf
import numpy as np
import pickle

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
with open('dataset.pkl', 'rb') as fopen:
    dataset = pickle.load(fopen)
dataset.keys()

dict_keys(['train_texts', 'test_texts', 'train_clss', 'test_clss', 'train_labels', 'test_labels'])

In [3]:
len(dataset['train_texts'])

73967

In [4]:
with open('dictionary.pkl', 'rb') as fopen:
    dictionary = pickle.load(fopen)

In [5]:
rev_dictionary = dictionary['rev_dictionary']
dictionary = dictionary['dictionary']

In [6]:
class Model:
    def __init__(self, size_layer, num_layers, embedded_size,
                 dict_size, learning_rate):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.float32, [None, None])
        self.mask = tf.placeholder(tf.int32, [None, None])
        self.clss = tf.placeholder(tf.int32, [None, None])
        mask = tf.cast(self.mask, tf.float32)
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
        outputs, _ = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded, dtype = tf.float32)
        outputs = tf.gather(outputs, self.clss, axis = 1, batch_dims = 1)
        self.logits = tf.layers.dense(outputs, 1)
        self.logits = tf.squeeze(self.logits, axis=-1)
        self.logits = self.logits * mask
        crossent = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.logits, labels=self.Y)
        crossent = crossent * mask
        crossent = tf.reduce_sum(crossent)
        total_size = tf.reduce_sum(mask)
        self.cost = tf.div_no_nan(crossent, total_size)
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        
        l = tf.round(tf.sigmoid(self.logits))
        self.accuracy = tf.reduce_mean(tf.cast(tf.boolean_mask(l, tf.equal(self.Y, 1)), tf.float32))

In [7]:
size_layer = 256
num_layers = 2
embedded_size = 256
learning_rate = 1e-3
batch_size = 128
epoch = 20

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,num_layers,embedded_size,len(dictionary),learning_rate)
sess.run(tf.global_variables_initializer())

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
UNK = 3

def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [10]:
train_X = str_idx(dataset['train_texts'], dictionary)
test_X = str_idx(dataset['test_texts'], dictionary)

In [11]:
train_clss = dataset['train_clss']
test_clss = dataset['test_clss']
train_Y = dataset['train_labels']
test_Y = dataset['test_labels']

In [12]:
batch_x, _ = pad_sentence_batch(train_X[:5], 0)
batch_y, _ = pad_sentence_batch(train_Y[:5], 0)
batch_clss, _ = pad_sentence_batch(train_clss[:5], -1)
batch_clss = np.array(batch_clss)
batch_mask = 1 - (batch_clss == -1)
batch_clss[batch_clss == -1] = 0

feed = {model.X: batch_x,
        model.Y: batch_y,
        model.mask: batch_mask,
        model.clss: batch_clss}
acc, loss, _ = sess.run([model.accuracy, model.cost,model.optimizer], feed_dict = feed)
acc, loss

(0.27272728, 0.68941796)

In [13]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x, _ = pad_sentence_batch(train_X[i : index], 0)
        batch_y, _ = pad_sentence_batch(train_Y[i : index], 0)
        batch_clss, _ = pad_sentence_batch(train_clss[i : index], -1)
        batch_clss = np.array(batch_clss)
        batch_mask = 1 - (batch_clss == -1)
        batch_clss[batch_clss == -1] = 0
        feed = {model.X: batch_x,
                model.Y: batch_y,
                model.mask: batch_mask,
                model.clss: batch_clss}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x, _ = pad_sentence_batch(test_X[i : index], 0)
        batch_y, _ = pad_sentence_batch(test_Y[i : index], 0)
        batch_clss, _ = pad_sentence_batch(test_clss[i : index], -1)
        batch_clss = np.array(batch_clss)
        batch_mask = 1 - (batch_clss == -1)
        batch_clss[batch_clss == -1] = 0
        feed = {model.X: batch_x,
                model.Y: batch_y,
                model.mask: batch_mask,
                model.clss: batch_clss}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 0/578 [00:00<?, ?it/s]

epoch 1, training avg loss 0.262667, training avg acc 0.000085
epoch 1, testing avg loss 0.252563, testing avg acc 0.000000


minibatch loop:   0%|          | 0/578 [00:00<?, ?it/s]

epoch 2, training avg loss 0.252687, training avg acc 0.000000
epoch 2, testing avg loss 0.250887, testing avg acc 0.000000


minibatch loop:   0%|          | 0/578 [00:00<?, ?it/s]

...

epoch 18, training avg loss 0.081986, training avg acc 0.674612
epoch 18, testing avg loss 0.546543, testing avg acc 0.129065


minibatch loop:   0%|          | 0/578 [00:00<?, ?it/s]

epoch 19, training avg loss 0.074648, training avg acc 0.702221
epoch 19, testing avg loss 0.547588, testing avg acc 0.135218


minibatch loop: 100%|██████████| 145/145 [02:01<00:00,  1.19it/s, accuracy=0.174, cost=0.512] 

epoch 20, training avg loss 0.068429, training avg acc 0.724982
epoch 20, testing avg loss 0.577269, testing avg acc 0.130517


In [ ]:
from tensor2tensor.utils import rouge
from tensorflow.keras.preprocessing import sequence

def calculate_rouges(predicted, batch_y):
    non = np.count_nonzero(batch_y, axis = 1)
    o = []
    for n in non:
        o.append([True for _ in range(n)])
    b = sequence.pad_sequences(o, dtype = np.bool, padding = 'post', value = False)
    batch_y = np.array(batch_y)
    rouges = []
    for i in range(predicted.shape[0]):
        a = batch_y[i][b[i]]
        p = predicted[i][b[i]]
        rouges.append(rouge.rouge_n([p], [a]))
    return np.mean(rouges)

In [55]:
batch_x, _ = pad_sentence_batch(test_X[: 5], 0)
batch_y, _ = pad_sentence_batch(test_Y[: 5], 0)
batch_clss, _ = pad_sentence_batch(test_clss[: 5], -1)
batch_clss = np.array(batch_clss)
batch_y = np.array(batch_y)
batch_x = np.array(batch_x)
cp_batch_clss = batch_clss.copy()
batch_mask = 1 - (batch_clss == -1)
batch_clss[batch_clss == -1] = 0

feed = {model.X: batch_x,
        model.mask: batch_mask,
        model.clss: batch_clss}
predicted = sess.run(tf.round(tf.sigmoid(model.logits)), feed_dict = feed)

In [59]:
from tensor2tensor.utils import rouge

def calculate_rouge(predicted, batch_y, cp_batch_clss, batch_x):
    f, y_, predicted_ = [], [], []
    for i in range(len(cp_batch_clss)):
        f.append(cp_batch_clss[i][cp_batch_clss[i] != -1])
        y_.append(batch_y[i][cp_batch_clss[i] != -1])
        predicted_.append(predicted[i][cp_batch_clss[i] != -1])
    
    actual, predict = [], []
    for i in range(len(f)):
        actual_, predict_ = [], []
        for k in range(len(f[i])):
            if k == (len(f[i]) - 1):
                s = batch_x[i][f[i][k]:]
                s = s[s != 0]
            else:
                s = batch_x[i][f[i][k]: f[i][k + 1]]
            s = [w for w in s if w not in [0, 1, 2, 3, 5, 6, 7, 8]]
            if y_[i][k]:
                actual_.extend(s)
            if predicted_[i][k]:
                predict_.extend(s)
        actual.append(actual_)
        predict.append(predict_)
    return rouge.rouge_n(predict, actual)

calculate_rouge(predicted, batch_y, cp_batch_clss, batch_x)

0.20137092

In [62]:
from tqdm import tqdm as tqdm_base
def tqdm(*args, **kwargs):
    if hasattr(tqdm_base, '_instances'):
        for instance in list(tqdm_base._instances):
            tqdm_base._decr_instances(instance)
    return tqdm_base(*args, **kwargs)

In [63]:
rouges = []

pbar = tqdm(
    range(0, len(test_X), 32), desc = 'minibatch loop')
for i in pbar:
    index = min(i + batch_size, len(test_X))
    batch_x, _ = pad_sentence_batch(test_X[i: index], 0)
    batch_y, _ = pad_sentence_batch(test_Y[i: index], 0)
    batch_clss, _ = pad_sentence_batch(test_clss[i: index], -1)
    batch_clss = np.array(batch_clss)
    batch_y = np.array(batch_y)
    batch_x = np.array(batch_x)
    cp_batch_clss = batch_clss.copy()
    batch_mask = 1 - (batch_clss == -1)
    batch_clss[batch_clss == -1] = 0

    feed = {model.X: batch_x,
            model.mask: batch_mask,
            model.clss: batch_clss}
    predicted = sess.run(tf.round(tf.sigmoid(model.logits)), feed_dict = feed)
    rouge_ = calculate_rouge(predicted, batch_y, cp_batch_clss, batch_x)
    rouges.append(rouge_)
    pbar.set_postfix(rouge = rouge_)


minibatch loop: 100%|██████████| 578/578 [10:19<00:00,  1.07s/it, rouge=0.155]


In [64]:
np.mean(rouges)

0.1613218